In [1]:
import re
import os
import random

# Set the seed for reproducibility
random.seed(42)
folder_name="train"
type = "edited"
zero_txt = f"/home/grads/r/rohan.chaudhury/Disfluency/treebank_transformations/brown/model_files/fluent.txt"
one_text = f"/home/grads/r/rohan.chaudhury/Disfluency/treebank_transformations/brown/model_files/disfluent.txt"


zero_texts = open(zero_txt).read().split('\n')
one_texts = open(one_text).read().split('\n')





# Ensure that both lists are of equal length
assert len(zero_texts) == len(one_texts)

print (len(zero_texts))
print (len(one_texts))

print ("No.of seps in fluent", sum([i.count("<SEP>") for i in zero_texts]))
print ("No.of seps in disfluent", sum([i.count("<SEP>") for i in one_texts]))
assert sum([i.count("<SEP>") for i in zero_texts])==sum([i.count("<SEP>") for i in one_texts])


def cleaning_function(input_texts):
    k=0
    input_texts_new=[]
    for i in range(len(input_texts)):
        k=k+1
        matched = re.search('c[a-z][0-9]+\.txt:(.*)', input_texts[i])
        try:
            temp = matched.group(1)
            # print (temp)
            temp = re.sub('\s+', ' ', temp)
            temp = temp.strip()
            input_texts_new.append(temp)
        except:
            print ("ERROR",input_texts[i])
            # print (k)
    return input_texts_new

zero_texts=cleaning_function(zero_texts)
one_texts=cleaning_function(one_texts)


print (len(zero_texts))
print (len(one_texts))


num_4 =10
print (zero_texts[num_4])
print (one_texts[num_4])

#Count no. of <SEP> in each list
print (sum([i.count("<SEP>") for i in zero_texts]))
print (sum([i.count("<SEP>") for i in one_texts]))
assert sum([i.count("<SEP>") for i in zero_texts])==sum([i.count("<SEP>") for i in one_texts])

zero_texts_extended = [zero_texts[i].split("<SEP>") for i in range(len(zero_texts))]
one_texts_extended = [one_texts[i].split("<SEP>") for i in range(len(one_texts))]


zero_texts_extended=[j.strip() for i in zero_texts_extended for j in i]
one_texts_extended=[j.strip() for i in one_texts_extended for j in i]


print (len(zero_texts_extended))
print (len(one_texts_extended))

assert len(zero_texts_extended)==len(one_texts_extended)
zero_texts_final=[]
one_texts_final=[]

def remove_unwanted_quotes(s):
    return re.sub(r"(^|[^a-zA-Z])'|'([^a-zA-Z]|$)", r'\1\2', s)

def remove_non_english_chars(text):
    # This regex retains English letters, digits, common punctuation, and whitespace.
    # Any character not in this set will be replaced with an empty string.
    pattern = r"[^a-zA-Z0-9\s.,!?@#$%&*()-_+=<>\"']"
    return re.sub(pattern, '', text)


def to_lowercase(match):
    return match.group(0).lower()

# result = re.sub(r'\b(?!(?:I\b|I\'[a-zA-Z]+|[A-Z]{2,}|[a-zA-Z])\b)[a-zA-Z]+\b', to_lowercase, text)


word_min = 0

def remove_non_letters_from_start(sentence):
    # Find the index where the first letter is
    try:
        sentence = remove_non_english_chars(sentence)
        sentence = remove_unwanted_quotes(sentence).strip()
        for idx, char in enumerate(sentence):
            if char.isalpha():
                break
        if idx!=0:
            print ("Error",sentence)
        # Return the sentence starting from the first letter
        text = sentence[idx:]
        text = re.sub('[.,:;`!()%#\/]', '', text)
        text = re.sub("''", "", text)
        
        # Remove partial words followed by spaces or '.'
        text = re.sub(r'\b\w+-([ .]|$)', '', text)
        text = re.sub('-', '', text)
        text = re.sub(r' +',' ', text)
        text = re.sub(r'\b(?!(?:I\b|I\'[a-zA-Z]+)\b)[a-zA-Z]+\b', to_lowercase, text.strip())
    except:
        text = sentence

    return text



def clean_more(text):
    text = re.sub('-LRB-|-RRB-|--', '', text)
    text = re.sub(' +', ' ', text).strip()
    text = text.replace(' .', '.')
    return text

k=0
for a_0,a_1 in zip(zero_texts_extended,one_texts_extended):
    if len(re.sub("\s+","",a_1.strip()))>-1 and a_0.strip()!=a_1.strip():
        zero_texts_final.append(remove_non_letters_from_start(clean_more(a_0)))
        one_texts_final.append(remove_non_letters_from_start(clean_more(a_1)))

    else:
        k=k+1
        # if len(a_1.strip())>1:
        #     print("Error in 1",a_1)
        # if len(a_2.strip())>1:
        #     print("Error in 2",a_2)
        # if len(a_3.strip())>1:
        #     print("Error in 3",a_3)
        # if len(a_4.strip())>1:
        #     print("Error in 4",a_4)
        # if len(a_5.strip())>1:
        #     print("Error in 5",a_5)
        # if len(a_6.strip())>1:
        #     print("Error in 6",a_6)
        # if len(a_7.strip())>1:
        #     print("Error in 7",a_7)
        # if len(a_8.strip())>1:
        #     print("Error in 8",a_8)
        # if len(a_9.strip())>1:
        #     print("Error in 9",a_9)
        # if len(a_10.strip())>1:
        #     print("Error in 10",a_10)
print ("no of errors",k)
threshold = 50

def merge_small_sentences(lst1, lst2, threshold=30):
    if len(lst1[0].strip())<threshold or len(lst2[0].strip())<threshold or lst1[0].strip()==lst2[0].strip():
        lst1[1] = lst1[0] + ' ' + lst1[1]
        lst2[1] = lst2[0] + ' ' + lst2[1]
        lst1.pop(0)
        lst2.pop(0)
    i = 1  # Start from 1 since we might need to merge with the previous sentence.
    while i < len(lst1):
        if len(lst1[i].strip()) < threshold or len(lst2[i].strip()) < threshold or lst1[i].strip()==lst2[i].strip():
            lst1[i-1] += ' ' + lst1[i]
            lst2[i-1] += ' ' + lst2[i]
            lst1.pop(i)
            lst2.pop(i)
            # Don't increment i since we shortened the list.
        else:
            i += 1
    lst1 = [re.sub(r' +', ' ', i.strip()) for i in lst1]
    lst2 = [re.sub(r' +', ' ', i.strip()) for i in lst2]
    print ("Minimum length",min([len(i) for i in lst1]),min([len(i) for i in lst2]))
    return lst1, lst2

print (len(zero_texts_final))
print (len(one_texts_final))

while min([len(i) for i in zero_texts_final])<threshold or min([len(i) for i in one_texts_final])<threshold:
    zero_texts_final,one_texts_final = merge_small_sentences(zero_texts_final,one_texts_final,threshold)
print ("Minimum length",min([len(i) for i in zero_texts_final]),min([len(i) for i in one_texts_final]))

print (len(zero_texts_final))
print (len(one_texts_final))


assert len(zero_texts_final)==len(one_texts_final)
# assert 1==2
num_1 = 100
num_2 = 50
num_3 = 125

print (zero_texts_final[num_3])
print (one_texts_final[num_3])


print ("-------------")
 
print (zero_texts_final[-1])
print (one_texts_final[-1])


print ("-------------")

print (zero_texts_final[num_1])
print (one_texts_final[num_1])

        
print (max([len(i.split(" ")) for i in zero_texts_final]))
print (max([len(i.split(" ")) for i in one_texts_final]))



print (min([len(i.split(" ")) for i in zero_texts_final]))
print (min([len(i.split(" ")) for i in one_texts_final]))


import numpy as np
from numpy import argmin
from numpy import argmax
print (zero_texts_final[argmin([len(i.split(" ")) for i in zero_texts_final])])
print (one_texts_final[argmin([len(i.split(" ")) for i in one_texts_final])])

maximum_1 =zero_texts_final[argmax([len(i) for i in zero_texts_final])]
maximum_2= one_texts_final[argmax([len(i) for i in one_texts_final])]
from transformers import T5Tokenizer
#calculate the number of tokens from T5 tokenizer
t5_pretrained_model_name = "t5-base"
model_max_length = 1024
t5_tokenizer = T5Tokenizer.from_pretrained(t5_pretrained_model_name, model_max_length=model_max_length)

print ("Maxinum_1")
question_tokens = t5_tokenizer(maximum_1, truncation=False, return_tensors="pt", padding=False)
print(len(question_tokens["input_ids"].squeeze()))
print ("Maxinum_2")
question_tokens = t5_tokenizer(maximum_2, truncation=False, return_tensors="pt", padding=False)
print(len(question_tokens["input_ids"].squeeze()))


maximum_1 =zero_texts_final[argmax([len(i.split(" ")) for i in zero_texts_final])]
maximum_2= one_texts_final[argmax([len(i.split(" ")) for i in one_texts_final])]
from transformers import T5Tokenizer
#calculate the number of tokens from T5 tokenizer
t5_pretrained_model_name = "t5-base"
model_max_length = 1024
t5_tokenizer = T5Tokenizer.from_pretrained(t5_pretrained_model_name, model_max_length=model_max_length)

print ("Maxinum_1")
question_tokens = t5_tokenizer(maximum_1, truncation=False, return_tensors="pt", padding=False)
print(len(question_tokens["input_ids"].squeeze()))
print ("Maxinum_2")
question_tokens = t5_tokenizer(maximum_2, truncation=False, return_tensors="pt", padding=False)
print(len(question_tokens["input_ids"].squeeze()))


print (argmin([len(i) for i in zero_texts_final]))
print (argmin([len(i) for i in one_texts_final]))


print (zero_texts_final[argmin([len(i) for i in zero_texts_final])])
print (one_texts_final[argmin([len(i) for i in one_texts_final])])


# print (argmin([len(i) for i in zero_texts_final]))
# print (argmin([len(i) for i in one_texts_final]))


directory_to_save = f"/home/grads/r/rohan.chaudhury/Disfluency/Generating-Fluent-Text-through-Curriculum-Learning-And-Disfluency-Augmentation/formatted_data/brown/{folder_name}/"
os.makedirs(directory_to_save, exist_ok=True)


with open(directory_to_save+"fluent.txt", 'w') as f:
    f.write('\n'.join(zero_texts_final))
with open(directory_to_save+"disfluent.txt", 'w') as f:
    f.write('\n'.join(one_texts_final))

print ("Done")



193
193
No.of seps in fluent 11329
No.of seps in disfluent 11329
ERROR 
ERROR 
192
192
Summer the gambling houses were closed despite the threats of Pierre Ameaux a gaming-card manufacturer. <SEP> Dancing was no longer permitted in the streets. The Bordel and other places of prostitution were emptied. <SEP> The slit breeches had to go. Drunkenness was no longer tolerated. <SEP> In defiance a chinless reprobate Jake Camaret marched down the aisle in St. Peters one Sunday morning followed by one of the women from the Bordel whose dress and walk plainly showed the lack of any shame. Plunking themselves down on the front bench they turned to smirk at those around them. <SEP> Johns first impulse was to denounce their blasphemy. The thought occurred that God would want this opportunity used to tell them about Him. <SEP> Calmly he opened the Bible and read of the woman at the well. He finished the worship service as if there had been no brazen attempt to dishonor God and man. <SEP> The next m

/home/grads/r/rohan.chaudhury/miniconda3/envs/multidoc2dial/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Maxinum_1
241
Maxinum_2
253
Maxinum_1
241
Maxinum_2
253
368
1154
his eyes burned feverishly he muttered impatiently
oh thats all right he said youre going to be a regular
Done
